In [1]:
import csv
import json
import pandas as pd
from datetime import datetime

vendorMaster = {}
with open('/Users/patelrudra/Documents/BOOKS/VendorsMaster.csv', mode ='r')as file:
    masterFile = csv.reader(file)    
    for vendor in masterFile:
        vendorMaster[vendor[1]] = vendor[0]
        
vendorItem = {}
with open('/Users/patelrudra/Documents/BOOKS/Item.csv', mode ='r')as file1:
    masterFile = csv.reader(file1)    
    for vendor in masterFile:
        vendorItem[vendor[1]] = vendor[0]
        
gstConfigurationData = {
    "0" : "1357403000000016427",
    "5" : "1357403000000016433",
    "12" : "1357403000000016439",
    "18" : "1357403000000016445",
    "28" : "1357403000000016451"
}
        
with open('/Users/patelrudra/Documents/Medkart/wms/venodrcreditmay.csv', mode ='r')as file:

    csvFile = csv.reader(file)
    data_list = []
    null_voucher_dict = []
    data={}
    count=0
    Bill=True
    notItems=[]
    
    for entry in csvFile:
        if entry[2] == '':
            if entry[1] == 'Grand Total':
                continue;
#             if entry[1] not in vendorItem:
#                 notItems.append(entry[1])
#                 continue
            data1 = {
                'item_id': vendorItem[entry[1]],
                'rate':entry[12],
                "itc_eligibility":"eligible",
                "quantity":"1",
                "item_order":len(data_list)+1,
               "tax_id" : gstConfigurationData[entry[1].split("_")[1]]
             }
            
            data_list.append(data1)
        else:
            data['line_items'] = data_list
            null_voucher_dict.append(data)
            data_list=[]
            date_object = datetime.strptime(entry[0], "%d-%b-%y")
            formatted_date = date_object.strftime("%Y-%m-%d")

            data = {
                'date': formatted_date,
                'vendor_id': vendorMaster[entry[1]],
                "vendor_credit_number" : entry[2],
                  "tax_override": False,
                "gst_no":entry[3],
                "is_inclusive_tax": False,
                 "reference_invoice_type":"registered",
                "reference_number":entry[3]
             }
            
    for data in null_voucher_dict:
        json_data = json.dumps(data, indent=4)
        with open("sample.json", "a") as outfile:
            outfile.write(json_data + '\n')  

In [16]:
null_voucher_dict.pop(0)
null_voucher_dict


[{'date': '2023-05-01',
  'vendor_id': '1357403000004097389',
  'vendor_credit_number': '230HO1PR00000107',
  'tax_override': False,
  'gst_no': '24AHNPB6309B1ZV',
  'is_inclusive_tax': False,
  'reference_invoice_type': 'registered',
  'reference_number': 'TD02718',
  'line_items': [{'item_id': '1357403000004358245',
    'rate': '462.85',
    'itc_eligibility': 'eligible',
    'quantity': '1',
    'item_order': 1,
    'tax_id': '1357403000000016439'}]},
 {'date': '2023-05-01',
  'vendor_id': '1357403000004185325',
  'vendor_credit_number': '230HO1PR00000106',
  'tax_override': False,
  'gst_no': '24AAJCV1878M1ZP',
  'is_inclusive_tax': False,
  'reference_invoice_type': 'registered',
  'reference_number': '23SZ010237',
  'line_items': [{'item_id': '1357403000004360018',
    'rate': '342.78',
    'itc_eligibility': 'eligible',
    'quantity': '1',
    'item_order': 1,
    'tax_id': '1357403000000016439'}]},
 {'date': '2023-05-01',
  'vendor_id': '1357403000004107568',
  'vendor_credit_

In [17]:
import requests
import json
url = "https://www.zohoapis.in/books/v3/vendorcredits?organization_id=60022053195"
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Zoho-oauthtoken 1000.486b579fc80b8f43ae298fa18881863a.24e37dc51d0a48da1235a655ca0f86df'
}
failedData = []
for data in null_voucher_dict:
    payload = json.dumps(data)


    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code >= 200 and response.status_code < 300:  # Assuming a successful response has a status code of 200
        data = response.json()
        # Process the response data as needed
        # print(data)
    else:

        print(f"API request failed with status code {response.status_code}")
        response_content = response.content.decode('utf-8')
        failedData.append(data['vendor_credit_number'])
        print(response_content)

print("__________________________________")
print("Finished")
print("__________________________________")
print("failed records bill number : ")
print(failedData)

__________________________________
Finished
__________________________________
failed records bill number : 
[]
